<a href="https://colab.research.google.com/github/jp-prud/PDI/blob/main/Atividade_3_Filtragem_Espacial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Filtro gaussiano

In [ ]:
import cv2
import numpy as np
import scipy.stats as stats
from skimage.util import random_noise

image = cv2.imread('t1.jpg')

def filtro_gaussiano_manual(image, tamanho_mascara, desvio_padrao):
    altura, largura, _ = image.shape
    mascara = np.zeros((tamanho_mascara, tamanho_mascara))
    centro = tamanho_mascara // 2

    for y in range(tamanho_mascara):
        for x in range(tamanho_mascara):
            ky = y - centro
            kx = x - centro
            mascara[y, x] = stats.norm.pdf(np.sqrt(ky ** 2 + kx ** 2), scale=desvio_padrao)
    mascara /= mascara.sum()

    output = np.zeros_like(image)

    for y in range(altura):
        for x in range(largura):
            for c in range(3):
                soma = 0
                for ky in range(tamanho_mascara):
                    for kx in range(tamanho_mascara):
                        py = y - tamanho_mascara // 2 + ky
                        px = x - tamanho_mascara // 2 + kx
                        if py >= 0 and px >= 0 and py < altura and px < largura:
                            soma += image[py, px, c] * mascara[ky, kx]
                output[y, x, c] = int(soma)

    return output

img_ruido = random_noise(image, mode='s&p', amount=0.05)
img_ruido = (255 * img_ruido).astype(np.uint8)

smoothed_3x3_manual = filtro_gaussiano_manual(img_ruido, 3, 1)
smoothed_5x5_manual = filtro_gaussiano_manual(img_ruido, 5, 1)
smoothed_7x7_manual = filtro_gaussiano_manual(img_ruido, 7, 1)

cv2.imshow('Gaussiana Manual 3x3', smoothed_3x3_manual)
cv2.imshow('Gaussiana Manual 5x5', smoothed_5x5_manual)
cv2.imshow('Gaussiana Manual 7x7', smoothed_7x7_manual)
cv2.waitKey(0)
cv2.destroyAllWindows()

Filtro mediana manual

In [ ]:
import cv2
import numpy as np
from skimage.util import random_noise

image = cv2.imread('t1.jpg')

def filtro_mediana_manual(image, tamanho_mascara):
    altura, largura, _ = image.shape
    output = np.zeros_like(image)

    for y in range(altura):
        for x in range(largura):
            for c in range(3):
                valores = []
                for ky in range(tamanho_mascara):
                    for kx in range(tamanho_mascara):
                        py = y - tamanho_mascara // 2 + ky
                        px = x - tamanho_mascara // 2 + kx
                        if py >= 0 and px >= 0 and py < altura and px < largura:
                            valores.append(image[py, px, c])
                output[y, x, c] = np.median(valores)

    return output

img_ruido = random_noise(image, mode='s&p', amount=0.05)
img_ruido = (255 * img_ruido).astype(np.uint8)

smoothed_3x3_manual = filtro_mediana_manual(img_ruido, 3)
smoothed_5x5_manual = filtro_mediana_manual(img_ruido, 5)
smoothed_7x7_manual = filtro_mediana_manual(img_ruido, 7)

cv2.imshow('Imagem Original', img_ruido)
cv2.imshow('Mediana Manual 3x3', smoothed_3x3_manual)
cv2.imshow('Mediana Manual 5x5', smoothed_5x5_manual)
cv2.imshow('Mediana Manual 7x7', smoothed_7x7_manual)
cv2.waitKey(0)
cv2.destroyAllWindows()


Filtro média manual

In [ ]:
import cv2
import numpy as np
from skimage.util import random_noise
import matplotlib.pyplot as plt

image = cv2.imread('t1.jpg')

def filtro_media_manual(image, tamanho_mascara):
    altura, largura, _ = image.shape
    mascara = np.ones((tamanho_mascara, tamanho_mascara)) / (tamanho_mascara * tamanho_mascara)
    output = np.zeros_like(image)

    for y in range(altura):
        for x in range(largura):
            for c in range(3):
                soma = 0
                for ky in range(tamanho_mascara):
                    for kx in range(tamanho_mascara):
                        py = y - tamanho_mascara // 2 + ky
                        px = x - tamanho_mascara // 2 + kx
                        if py >= 0 and px >= 0 and py < altura and px < largura:
                            soma += image[py, px, c] * mascara[ky, kx]
                output[y, x, c] = int(soma)

    return output

img_ruido = random_noise(image, mode='s&p', amount=0.05)
img_ruido = (255 * img_ruido).astype(np.uint8)

img_3x3_suavizada = filtro_media_manual(img_ruido, 3)

img_5x5_suavizada = filtro_media_manual(img_ruido, 5)

img_7x7_suavizada = filtro_media_manual(img_ruido, 7)

cv2.imshow('Imagem Ruidosa', img_ruido)
cv2.imshow('Suavização Média 3x3', img_3x3_suavizada)
cv2.imshow('Suavização Média 5x5', img_5x5_suavizada)
cv2.imshow('Suavização Média 7x7', img_7x7_suavizada)
cv2.waitKey(0)
cv2.destroyAllWindows()